# Train and fine-tune an OCR model

I will be using this guide here: https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/TrOCR/Fine_tune_TrOCR_on_IAM_Handwriting_Database_using_Seq2SeqTrainer.ipynb

How to build my image dataset: https://huggingface.co/docs/datasets/en/image_dataset

I read this guide on setting the training parameters (see Seq2SeqTrainer part of the guide as well as the notebook) https://github.com/philschmid/document-ai-transformers/blob/main/training/donut_sroie.ipynb

In [1]:
!pip install evaluate jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 24.3 MB/s eta 0:00:0000:0100:01


In [2]:
from datasets import Dataset
import pandas as pd

images_df = pd.read_csv("images/metadata.csv")
images_ds = Dataset.from_pandas(images_df)
images_df

,file_name,text
0,addition_application.png,addition application coffee trade environment ...
1,case_week.png,case week group recognition transportation com...
2,combination_school.png,combination school responsibility average year...
3,concept_administration.png,concept administration sign device reaction so...
4,dealer_passenger.png,dealer passenger tension intention responsibil...
5,discussion_guidance.png,discussion guidance language interest light re...
6,efficiency_answer.png,efficiency answer chance ability intention tim...
7,engine_paper.png,engine paper intention category reaction incom...
8,hearing_winner.png,hearing winner book building importance improv...
9,manufacturer_care.png,manufacturer care page charity piano nothing r...


In [3]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

model_name = "microsoft/trocr-base-printed"
processor = TrOCRProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from PIL import Image
from pathlib import Path
import torch

def create_image_and_process_text(item):
    file_name = item["file_name"]
    text = item["text"]

    file_path = Path("images") / file_name

    image = Image.open(file_path).convert("RGB")
    
    pixel_values = processor(image, return_tensors="pt").pixel_values
    pixel_values = pixel_values.squeeze()

    labels = processor.tokenizer(text, padding="max_length", max_length=16).input_ids
    labels = torch.tensor(labels)

    encoding = { "pixel_values": pixel_values, "labels": labels }
    return encoding

In [5]:
inout_images_ds = images_ds.map(create_image_and_process_text, remove_columns=["file_name", "text"]) 

  0%|          | 0/24 [00:00<?, ?ex/s]

In [6]:
from operator import itemgetter

train_test_ds = inout_images_ds.train_test_split()
train_dataset, eval_dataset = itemgetter("train", "test")(train_test_ds)
train_dataset, eval_dataset

(Dataset({
     features: ['pixel_values', 'labels'],
     num_rows: 18
 }),
 Dataset({
     features: ['pixel_values', 'labels'],
     num_rows: 6
 }))

In [7]:
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

model.config.vocab_size = model.config.decoder.vocab_size
model.config.eos_token_id = processor.tokenizer.sep_token_id

# model.config.max_length = 16
# model.config.early_stopping = True
# model.config.no_repeat_ngram_size = 3
# model.config.length_penalty = 2.0
# model.config.num_beams = 4

In [8]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    num_train_epochs=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    # logging_steps=100,
    evaluation_strategy="epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    fp16=True,
    output_dir="./",
    # logging_steps=2,
    # save_steps=1000,
    # eval_steps=200,
    report_to="none",
)

In [9]:
from evaluate import load

cer_metric = load("cer")

In [10]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    print(pred_str, label_str)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return { "character_error_rate": cer }

In [11]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)

Using cuda_amp half precision backend


In [12]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18
  Num Epochs = 16
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 288


Epoch,Training Loss,Validation Loss,Character Error Rate
1,No log,4.881782,0.809249
2,No log,3.932956,0.728324
3,No log,3.328248,0.630058
4,No log,2.075616,0.549133
5,No log,2.274999,0.479769
6,No log,2.693207,0.462428
7,No log,1.408315,0.358382
8,No log,2.297039,0.381503
9,No log,1.063546,0.381503
10,No log,1.191752,0.213873


***** Running Evaluation *****
  Num examples = 6
  Batch size = 1


['wayway environment', ' environment environment environment', ' environment environment environment environment', 'way environment', 'way', 'way environment environment environment'] ['addition application coffee trade environment competition', 'discussion guidance language interest light region mall situation', 'supermarket chance insect feedback preference impression', 'standard knowledge trainer video theory term line penalty', 'possibility meat argument foundation force nothing honey', 'manufacturer care page charity piano nothing resolution']


***** Running Evaluation *****
  Num examples = 6
  Batch size = 1


...

['addition application coffee trade environment competition', 'discussion guidance language interest light region mall su', 'supermarket chance insert feedback preference impression', 'standard knowledge trainer video theory term time penalty', 'personality meat argument foundation force nothing honey', 'manufacturer care page charity plan no obligation'] ['addition application coffee trade environment competition', 'discussion guidance language interest light region mall situation', 'supermarket chance insect feedback preference impression', 'standard knowledge trainer video theory term line penalty', 'possibility meat argument foundation force nothing honey', 'manufacturer care page charity piano nothing resolution']


***** Running Evaluation *****
  Num examples = 6
  Batch size = 1


['addition application coffee trade environment competition', 'discussion guidance language interest light region mall su', 'supermarket chance insert feedback preference impression', 'standard knowledge trainer video theory term time penalty', 'personality meat argument foundation force nothing honey', 'manufacturer care page charity plan noHD election'] ['addition application coffee trade environment competition', 'discussion guidance language interest light region mall situation', 'supermarket chance insect feedback preference impression', 'standard knowledge trainer video theory term line penalty', 'possibility meat argument foundation force nothing honey', 'manufacturer care page charity piano nothing resolution']




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=288, training_loss=1.3577116860283747, metrics={'train_runtime': 511.9958, 'train_samples_per_second': 0.563, 'train_steps_per_second': 0.563, 'total_flos': 2.155061350068388e+17, 'train_loss': 1.3577116860283747, 'epoch': 16.0})

In [15]:
save_model = False
if save_model:
    model.save_pretrained("my-trocr-model", from_pt=True)
    processor.save_pretrained("my-trocr-model", from_pt=True)

Configuration saved in my-trocr-model/config.json
Model weights saved in my-trocr-model/pytorch_model.bin
Feature extractor saved in my-trocr-model/preprocessor_config.json
tokenizer config file saved in my-trocr-model/tokenizer_config.json
Special tokens file saved in my-trocr-model/special_tokens_map.json
